## Setup

Edit `ARBOR_LOCATION` and `ARBOR_BUILD_CATALOGUE` to match the right `__init__.py` file and `arbor-build-catalogue` binary. (or delete the entire setup thing)

In [ ]:
import time
import sys

# BEGIN SET UP ARBOR PATH
import importlib
ARBOR_LOCATION = '/specialhome/lennart/Repos/arbor-sim/arbor/build/python/arbor/__init__.py'
spec = importlib.util.spec_from_file_location('arbor', ARBOR_LOCATION)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module
spec.loader.exec_module(module)
# END SET UP ARBOR PATH

import iopublic
import numpy as np
import matplotlib.pyplot as plt
iopublic.ARBOR_BUILD_CATALOGUE = '/specialhome/lennart/Repos/arbor-sim/arbor/build/arbor-build-catalogue'

In [ ]:
selected = '2021-12-08-shadow_averages_0.01_0.8_d1666304-c6fc-4346-a55d-a99b3aad55be'

## Simple comparison

To recompile mechanism libraries and test simple behaviour, run cells below

In [ ]:
%%time
iopublic.SMOL_MODEL_DIR = 'smol_model'
t, vs1 = iopublic.simulate_tuned_network(selected, tfinal=100)

In [ ]:
%%time
iopublic.SMOL_MODEL_DIR = 'smol_model_nmlcc'
t, vs2 = iopublic.simulate_tuned_network(selected, tfinal=100)

In [ ]:
for i in range(0, 500, 50):
    plt.plot(t, vs1[i], color='k', label='jneuroml' if i == 0 else None)
    plt.plot(t, vs2[i], '--', color='k', label='nmlcc' if i == 0 else None)
plt.xlabel('time (ms)')
plt.ylabel('soma potential (mV)')
plt.legend()

## Plot timings

In [ ]:
out = []
for tfinal in range(0, 400, 100):
    iopublic.SMOL_MODEL_DIR = 'smol_model'
    a = time.monotonic()
    t1, vs1 = iopublic.simulate_tuned_network(selected, tfinal=tfinal)
    b = time.monotonic()
    iopublic.SMOL_MODEL_DIR = 'smol_model_nmlcc'
    c = time.monotonic()
    t2, vs2 = iopublic.simulate_tuned_network(selected, tfinal=tfinal)
    d = time.monotonic()
    out.append((tfinal, t1, vs1, t2, vs2, b - a, d - c))
    print(tfinal, b - a, d - c)

In [ ]:
tf, t1, t2 = np.array([(x[0], x[5], x[6]) for x in out]).T
plt.plot(tf, t1, 'o--', label='jneuroml+manual editing')
plt.plot(tf, t2, 'o--', label='nmlcc')
plt.legend()
plt.title('Full IO network simulation')
plt.xlabel('Biological time (ms)')
plt.ylabel('Simulation time (s)')
plt.ylim(0, plt.ylim()[1])